In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from learning_wavelets.data import im_generators, ConcatenateGenerators
from learning_wavelets.evaluate import Metrics
from learning_wavelets.keras_utils.thresholding import SoftThresholding
from learning_wavelets.learned_wavelet import learned_wavelet
from learning_wavelets.unet import unet
from learning_wavelets.wavelet_denoising import wavelet_denoising

Using TensorFlow backend.


In [4]:
np.random.seed(0)

In [5]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [6]:
grey = True
n_channels = 3
if grey:
    n_channels = 1
im_gen_train, im_gen_val, im_gen_test, size, n_samples_train = im_generators(
    'bsd68', 
    batch_size=1, 
    validation_split=0.1, 
    no_augment=True, 
    noise_std=30,
    grey=grey
)    
im_bsd68 = ConcatenateGenerators(im_gen_train, im_gen_val, im_gen_test)

In [7]:
all_net_params = [
    {
        'name': 'unet',
        'init_function': unet,
        'run_params': {
            'n_layers': 5, 
            'pool': 'max', 
            "layers_n_channels": [64, 128, 256, 512, 1024], 
            'layers_n_non_lins': 2,
            'non_relu_contract': False,
            'input_size': (size, size, n_channels),
        },
        'run_id': 'unet_div2k_30_1570805218',
    },
    {
        'name': 'unet_bn',
        'init_function': unet,
        'run_params': {
            'n_layers': 5, 
            'pool': 'max', 
            "layers_n_channels": [64, 128, 256, 512, 1024], 
            'layers_n_non_lins': 2,
            'non_relu_contract': False,
            'bn': True,
            'input_size': (size, size, n_channels),
        },
        'run_id': 'unet_div2k_30_1571047085',
    },
    {
        'name': 'learned wavelet',
        'init_function': learned_wavelet,
        'run_params': {
            'n_scales': 5, 
            'n_details': 256, 
            'n_coarse': 1, 
            'n_groupping': 256,
            'input_size': (size, size, n_channels),
        },
        'run_id': 'learned_wavelet_div2k_30_1570806527',
    },
    {
        'name': 'learned wavelet_st_0.1',
        'init_function': learned_wavelet,
        'run_params': {
            'n_scales': 5, 
            'n_details': 256, 
            'n_coarse': 1, 
            'n_groupping': 256,
            'denoising_activation': SoftThresholding(0.1),
            'input_size': (size, size, n_channels),
        },
        'run_id': 'learned_wavelet_div2k_30_1571043286',
    },
#     {
#         'name': 'learned wavelet_st_0.01',
#         'init_function': learned_wavelet,
#         'run_params': {
#             'n_scales': 5, 
#             'n_details': 256, 
#             'n_coarse': 1, 
#             'n_groupping': 256,
#             'denoising_activation': SoftThresholding(0.01),
#             'input_size': (size, size, n_channels),
#         },
#         'run_id': 'learned_wavelet_div2k_30_1571044355',
#     },
#     {
#         'name': 'learned wavelet_st_1',
#         'init_function': learned_wavelet,
#         'run_params': {
#             'n_scales': 5, 
#             'n_details': 256, 
#             'n_coarse': 1, 
#             'n_groupping': 256,
#             'denoising_activation': SoftThresholding(1.0),
#             'input_size': (size, size, n_channels),
#         },
#         'run_id': 'learned_wavelet_div2k_30_1571044678',
#     },
    {
        'name': 'learned wavelet_linear',
        'init_function': learned_wavelet,
        'run_params': {
            'n_scales': 5, 
            'n_details': 256, 
            'n_coarse': 1, 
            'n_groupping': 256,
            'denoising_activation': 'linear',
            'input_size': (size, size, n_channels),
        },
        'run_id': 'learned_wavelet_div2k_30_1571061344',
    },
]

In [8]:
def unpack_model(init_function=None, run_params=None, run_id=None, epoch=500, **dummy_kwargs):
    model = init_function(**run_params)
    chkpt_path = f'checkpoints/{run_id}-{epoch}.hdf5'
    model.load_weights(chkpt_path)
    return model

def enumerate_seq(seq, name):
    return (seq[i] for i in tqdm_notebook(range(len(seq)), desc=f'Val files for {name}'))

def metrics_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics()
    pred_and_gt = [
        (model.predict_on_batch(images_noisy), images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, name)
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos)
    return metrics

def metrics_original():
    metrics = Metrics()
    pred_and_gt = [
        (images_noisy, images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, 'Original noisy image')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Original noisy image'):
        metrics.push(images, im_recos)
    return metrics

def metrics_wavelets():
    metrics = Metrics()
    pred_and_gt = [
        (wavelet_denoising(images_noisy[0], 30/255, n_scales=5), images_gt[0])
        for images_noisy, images_gt in enumerate_seq(im_bsd68, 'Wavelet denoising')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Stats for wavelet denoising'):
        metrics.push(images, im_recos)
    return metrics

In [9]:
%%time
metrics = []
# for net_params in all_net_params:
#     metrics.append((net_params['name'], metrics_for_params(**net_params)))
    
metrics.append(('original', metrics_original()))
metrics.append(('wavelets', metrics_wavelets()))


CPU times: user 42.3 s, sys: 20.1 s, total: 1min 2s
Wall time: 1min 24s


In [10]:
metrics

[('original', PSNR = 19.3 +/- 1.086 SSIM = 0.3481 +/- 0.2035),
 ('wavelets', PSNR = 22.2 +/- 1.432 SSIM = 0.4579 +/- 0.1667)]

In [11]:
metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())

In [12]:
# import pickle
# with open('metrics_net_rec_fastmri', 'wb') as f:
#     pickle.dump(metrics, f)

In [13]:
def n_model_params_for_params(reco_function=None, val_gen=None, name=None, **net_params):
    model = unpack_model(**net_params)
    n_params = model.count_params()
    return n_params

In [18]:
%%time
n_params = {}
# for net_params in all_net_params:
#     n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
n_params['original'] =  0
n_params['wavelets'] =  2  # number of scales and type of wavelets

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


In [19]:
def runtime_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    data = im_bsd68[0]
    start = time.time()
    model.predict_on_batch(data[0])
    end = time.time()
    return end - start

In [20]:
%%time
runtimes = {}
# for net_params in tqdm_notebook(all_net_params):
#     runtimes[net_params['name']] =  runtime_for_params(**net_params)
    
runtimes['original'] = 'NA'
runtimes['wavelets'] = 'NA'  # TODO: code function for that

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


In [21]:
metrics_table = pd.DataFrame(
    index=[name for name, _ in metrics], 
    columns=['PSNR-mean (std) (dB)', 'SSIM-mean (std)', '# params', 'Runtime (s)'],
)
for name, m in metrics:
    metrics_table.loc[name, 'PSNR-mean (std) (dB)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['PSNR'].mean(), 
        std=m.metrics['PSNR'].stddev(),
    )
    metrics_table.loc[name, 'SSIM-mean (std)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['SSIM'].mean(), 
        std=m.metrics['SSIM'].stddev(),
    )
    metrics_table.loc[name, '# params'] = "{}".format(
        n_params[name], 
     )
    metrics_table.loc[name, 'Runtime (s)'] = "{runtime:.4}".format(
        runtime=runtimes[name], 
    )

In [22]:
metrics_table

,PSNR-mean (std) (dB),SSIM-mean (std),# params,Runtime (s)
original,19.3 (0.5428),0.3481 (0.1017),0,NA
wavelets,22.2 (0.7161),0.4579 (0.08334),2,NA
